In [1]:
from itertools import combinations_with_replacement
import pandas as pd
from config import paths, dataset, layout, quality_metrics
import optuna
from optuna.visualization import plot_contour
from optuna.distributions import IntDistribution, FloatDistribution


/Users/fuga_takata/dev/vdslab-project/hyperparameter_optimization/.venv/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
study = optuna.load_study(
    study_name="stress",
    storage=f"sqlite:////Users/fuga_takata/dev/vdslab-project/hyperparameter_optimization/data/SS/1138_bus/25a73bb2-ae88-4b4a-9f14-0acf4ee3b624/grid/data.sql",
)

len(study.trials)

8000

In [3]:
UUID = "25a73bb2-ae88-4b4a-9f14-0acf4ee3b624"
P_NAMES = ["number_of_pivots", "number_of_iterations", "eps"]
p_names_comb_list = list(combinations_with_replacement(P_NAMES, 2))

for layout_name in layout.LAYOUT_NAMES:
    for dataset_name in dataset.DATASET_NAMES:
        # if dataset_name != 'les_miserables':
        #     continue
        data_dir = paths.get_data_dir(
            layout_name=layout_name, dataset_name=dataset_name, uuid=UUID
        )
        grid_data_path = data_dir.joinpath("grid")
        if not grid_data_path.exists():
            continue

        for qm_name in quality_metrics.ALL_QM_NAMES:
            # study = optuna.create_study(
            #     study_name=qm_name,
            #     storage=f"sqlite:///{grid_data_path.resolve()}/data copy.sql",
            #     load_if_exists=True,
            #     direction=quality_metrics.QUALITY_METRICS_MAP[
            #         qm_name
            #     ].direction,
            # )
            study = optuna.load_study(
                study_name=qm_name,
                storage=f"sqlite:///{grid_data_path.resolve()}/data.sql",
            )
            fig = plot_contour(study=study)
            # fig.show()
            fig.write_image(f"images/{dataset_name}/{qm_name}.png")

In [4]:
from PIL import Image, ImageDraw, ImageFont
from pathlib import Path


def get_concat_h(im1, im2):
    dst = Image.new("RGB", (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst


def get_concat_v(im1, im2):
    dst = Image.new("RGB", (im1.width, im1.height + im2.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst


for layout_name in layout.LAYOUT_NAMES:
    for dataset_name in dataset.DATASET_NAMES:
        p = Path("images").joinpath(dataset_name)
        if not p.exists():
            continue
        dst_export_path = f"images/{dataset_name}/all.png"
        images = []
        tmp = []
        for qm_name in quality_metrics.ALL_QM_NAMES:
            image_path = f"images/{dataset_name}/{qm_name}.png"
            img = Image.open(image_path)
            draw = ImageDraw.Draw(img)

            width = img.width // 20
            height = img.height // 10
            font = ImageFont.truetype(font="Arial.ttf", size=width)
            draw.text(
                (width, height),
                f"{qm_name} {'+' if quality_metrics.QUALITY_METRICS_MAP[qm_name].direction == 'maximize' else '-'}",
                "black",
                font=font,
            )

            tmp.append({"image": img})

            if len(tmp) == 3:
                images.append(tmp)
                tmp = []

        dst = None
        for v in images:
            h_dst = None
            for h in v:
                if h_dst is None:
                    h_dst = h["image"]
                    continue
                h_dst = get_concat_h(h_dst, h["image"])
            if dst is None:
                dst = h_dst
                continue
            dst = get_concat_v(dst, h_dst)
            # dst = h_dst

        dst.save(dst_export_path)

In [5]:
# UUID = "25a73bb2-ae88-4b4a-9f14-0acf4ee3b624"
# P_NAMES = ["number_of_pivots", "number_of_iterations", "eps"]
# p_names_comb_list = list(combinations_with_replacement(P_NAMES, 2))

# for layout_name in layout.LAYOUT_NAMES:
#     for dataset_name in dataset.DATASET_NAMES:
#         if dataset_name != "les_miserables":
#             continue
#         data_dir = paths.get_data_dir(
#             layout_name=layout_name, dataset_name=dataset_name, uuid=UUID
#         )
#         grid_data_path = data_dir.joinpath("grid")
#         if not grid_data_path.exists():
#             continue

#         df = pd.DataFrame()
#         # temp_df = pd.DataFrame()
#         # for i in range(4):
#         #     temp_df = pd.concat([temp_df, pd.read_pickle(grid_data_path.joinpath(f"20split-{i}.pkl"))])
#         for i in range(4):
#             df = pd.concat(
#                 [
#                     df,
#                     pd.read_pickle(
#                         grid_data_path.joinpath(f"20split-{i}.pkl")
#                     ),
#                 ]
#             )
#         for row in df.itertuples():
#             print(row.params)

In [6]:
# UUID = "cd7c1537-2ebe-4434-9ead-f7b1a866815a"
# P_NAMES = ["number_of_pivots", "number_of_iterations", "eps"]
# p_names_comb_list = list(combinations_with_replacement(P_NAMES, 2))

# for layout_name in layout.LAYOUT_NAMES:
#     for dataset_name in dataset.DATASET_NAMES:
#         # if dataset_name != "les_miserables":
#         #     continue
#         data_dir = paths.get_data_dir(
#             layout_name=layout_name, dataset_name=dataset_name, uuid=UUID
#         )
#         grid_data_path = data_dir.joinpath("grid")
#         if not grid_data_path.exists():
#             continue

#         df = pd.DataFrame()
#         # temp_df = pd.DataFrame()
#         # for i in range(4):
#         #     temp_df = pd.concat([temp_df, pd.read_pickle(grid_data_path.joinpath(f"20split-{i}.pkl"))])
#         for i in range(4):
#             df = pd.concat(
#                 [
#                     df,
#                     pd.read_pickle(
#                         grid_data_path.joinpath(f"20split-{i}.pkl")
#                     ),
#                 ]
#             )

#         for qm_name in quality_metrics.ALL_QM_NAMES:
#             study = optuna.create_study(
#                 study_name=qm_name,
#                 storage=f"sqlite:///{grid_data_path.resolve()}.sql",
#             )
#             for i, row in df.iterrows():
#                 d = row.to_dict()
#                 params = d["params"]
#                 if "edge_length" in params:
#                     del params["edge_length"]

#                 qualities = d["qualities"]

#                 trial = optuna.trial.create_trial(
#                     params=params,
#                     value=qualities[qm_name],
#                     distributions={
#                         "number_of_pivots": IntDistribution(5, 100),
#                         "number_of_iterations": IntDistribution(10, 200),
#                         "eps": FloatDistribution(0.05, 1.0),
#                     },
#                 )
#                 study.add_trial(trial)
#             # fig = plot_contour(study=study)
#             # fig.write_image(f"images/{dataset_name}/{qm_name}.png")